<a href="https://colab.research.google.com/github/psytky03/Analysis_MISC/blob/master/spliceAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objectif 
Get all probabilities of splicing impact along a transcript.

## How 
Create a vcf file with all possible snps of a transcript defined by the user. 
Then run SpliceAI and export a bedgraph to display probabilites on IGV

https://github.com/Illumina/SpliceAI

In [1]:
# Download hg19.fa and refGene
!wget http://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz
!wget http://hgdownload.soe.ucsc.edu/goldenPath/hg19/database/refGene.txt.gz

--2020-05-26 10:39:57--  http://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 948731419 (905M) [application/x-gzip]
Saving to: ‘hg19.fa.gz’

hg19.fa.gz          100%[===================>] 904.78M  14.7MB/s    in 58s     

2020-05-26 10:40:55 (15.6 MB/s) - ‘hg19.fa.gz’ saved [948731419/948731419]

--2020-05-26 10:40:57--  http://hgdownload.soe.ucsc.edu/goldenPath/hg19/database/refGene.txt.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7751489 (7.4M) [application/x-gzip]
Saving to: ‘refGene.txt.gz’

refGene.txt.gz      100%[===================>]   7.39M  1.53MB/s

In [0]:
!gzip -d hg19.fa.gz

In [3]:
# Install dependencies
!pip install refgene_parser
!pip install pyfastx
!pip install spliceai
!pip install biopython

  Created wheel for refgene-parser: filename=refgene_parser-0.0.1-cp36-none-any.whl size=4786 sha256=dbd3b3caa311d85f75f0a0bcfec79285d63e2e23018d5833fa9227e93f7f17da
  Stored in directory: /root/.cache/pip/wheels/44/65/1a/1ff7df49af86cb3acf3fbea1fe7d768e600d800c504fa21dd3
Successfully built refgene-parser
     |████████████████████████████████| 839kB 19.9MB/s 
     |████████████████████████████████| 16.7MB 211kB/s 
     |████████████████████████████████| 10.8MB 46.3MB/s 
  Created wheel for pyfaidx: filename=pyfaidx-0.5.8-cp36-none-any.whl size=25051 sha256=dbd9650276ec0c9f860e56b36080dba1741c911824f9d802a84e22c4e07c7dc1
  Stored in directory: /root/.cache/pip/wheels/58/ea/ee/59d4649b0fb82a0690bdeae834bc85891b306126bcc067e29f
Successfully built pyfaidx
     |████████████████████████████████| 2.3MB 21.4MB/s 


In [0]:
from refgene_parser import RefGene
import pyfastx

In [0]:
refgene = RefGene("refGene.txt.gz")
hg19 = pyfastx.Fasta('hg19.fa' )

In [0]:
def mute(ref):
  """If I give you A, return me [C,G,T]"""
  if ref not in "ACGT":
    return None
  return list("ACGT".replace(ref,""))


In [0]:
gene = refgene.gene_by_id("NM_003122") # <==== Put your transcript name as defined in refgene

margin = 0 # <=== You can add a left/right margin around your transcript

# Get sequence of the transcript
seq = hg19.fetch(gene.chrom, (gene.start - margin, gene.end + margin))

# Get first position according margin
pos = gene.start - margin 

# Create a VCF file 
with open("variants.vcf", "w") as file:

  print("""##fileformat=VCFv4.2
##fileDate=20191004
##reference=GRCh37/hg19
##contig=<ID=chr1,length=249250621>
##contig=<ID=chr2,length=243199373>
##contig=<ID=chr3,length=198022430>
##contig=<ID=chr4,length=191154276>
##contig=<ID=chr5,length=180915260>
##contig=<ID=chr6,length=171115067>
##contig=<ID=chr7,length=159138663>
##contig=<ID=chr8,length=146364022>
##contig=<ID=chr9,length=141213431>
##contig=<ID=chr10,length=135534747>
##contig=<ID=chr11,length=135006516>
##contig=<ID=chr12,length=133851895>
##contig=<ID=chr13,length=115169878>
##contig=<ID=chr14,length=107349540>
##contig=<ID=chr15,length=102531392>
##contig=<ID=chr16,length=90354753>
##contig=<ID=chr17,length=81195210>
##contig=<ID=chr18,length=78077248>
##contig=<ID=chr19,length=59128983>
##contig=<ID=chr20,length=63025520>
##contig=<ID=chr21,length=48129895>
##contig=<ID=chr22,length=51304566>
##contig=<ID=chrX,length=155270560>
##contig=<ID=chrY,length=59373566>
#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO""", file = file)

  # Loop over all base in sequence, and create 3 alternatives mutation for each one
  for base in seq:
    for alt in mute(base):
      print(gene.chrom, pos, ".", base, alt, 30, "PASS",sep="\t", file = file )
    pos += 1 
    


Run Splice AI.. This can takes a while if you transcript is long .. You can enable GPU from notebook Settings.
But at the end, prefer running this notebook on your own server . 

In [0]:
!spliceai -I variants.vcf -O output.vcf -R hg19.fa -A grch37

Using TensorFlow backend.
2020-05-26 06:43:38.537850: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-26 06:44:14.104919: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-26 06:44:14.163014: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-26 06:44:14.163641: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-05-26 06:44:14.163696: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-26 06:44:14.419368: I tensorflow/stream

In [0]:
# Create a bedgraph.. The following line export first element of spliceAI output ( DS_AG )
# There is 3 snps per position. Change modulo values to keep only once : NR%3 == 0
!echo "track type=bedGraph name=\"BedGraph Format\" description=\"BedGraph format\" visibility=full color=200,100,0 altColor=0,100,200 priority=20" > proba.bedGraph
!cat output.vcf|grep Splice|sed "s/|/\t/g"|awk 'BEGIN{OFS="\t"}NR%3 == 0{print $1,$2,$2+1, $10}' >> proba.bedGraph

track type=bedGraph name="BedGraph Format" description="BedGraph format" visibility=full color=200,100,0 altColor=0,100,200 priority=20
